In [1]:
import scipy.stats as sts
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from functools import partial

import warnings

warnings.filterwarnings("ignore")

np.set_printoptions(threshold=5, suppress=True, linewidth=10**3)
sns.set(style="whitegrid")

In [ ]:
mu, sig, lam, dt = 0, 1, 10, 0.1
n_obs, samples, seed = 10**3, 25, 25

In [ ]:
u_grid = np.linspace(0, 10, n_obs)

print(u_grid.shape)
u_grid[:10]

In [ ]:
Deltas = (np.ones(shape=n_obs + 1) * dt) * np.arange(n_obs + 1)

print(Deltas.shape)
Deltas[:12]

### Brownian motion

Note that $B_{k \Delta} = \sum\limits_{j=1}^k \left[ B_{j \Delta} - B_{(j - 1) \Delta} \right]$, where $B_{j \Delta} - B_{(j - 1)} \sim N(0, \; \Delta).$

In [ ]:
np.random.seed(seed)

W_increments = sts.norm.rvs(loc=0, scale=np.sqrt(dt), size=(samples, n_obs))
W_path = np.cumsum(W_increments, axis=1)
W_path = np.insert(W_path, 0, 0, axis=1)

print(W_path.shape)
W_path

In [ ]:
plt.figure(figsize=(14, 7))
plt.grid(True)
for i in range(samples):
    plt.plot(Deltas, W_path[i])

plt.xlabel('Time')
plt.ylabel('Brownian motion')

plt.savefig('Brownian motion')
plt.show()

### Poisson process

Similarly $N_{k \Delta} = \sum\limits_{j=1}^{k} \left[ N_{j \Delta} - N_{(j - 1) \Delta} \right]$, where $N_{j \Delta} - N_{(j - 1) \Delta} \sim Pois(\lambda \Delta)$.

In [ ]:
np.random.seed(seed)

N_increments = sts.poisson.rvs(mu=lam * dt, size=(samples, n_obs))
N_path = np.cumsum(N_increments, axis=1)
N_path = np.insert(N_path, 0, 0, axis=1)

print(N_path.shape)
N_path

In [ ]:
plt.figure(figsize=(14, 7))
plt.grid(True)
for i in range(samples):
    plt.step(Deltas[:10], N_path[i][:10], where="post")

plt.xlabel('Time')
plt.ylabel('Poisson process')

plt.savefig('Poisson process')
plt.show()

### Compound Poisson process

In [ ]:
np.random.seed(seed)

chi_seq = sts.norm.rvs(loc=0, scale=1, size=(samples, np.max(N_path[:, -1])))
chi_sum = np.cumsum(chi_seq, axis=1)
chi_sum = np.insert(chi_sum, 0, 0, axis=1)

CPP_path = np.array([np.array([chi_sum[i][n_t] for n_t in N_path[i]]) for i in range(samples)])

print(CPP_path.shape)
CPP_path

In [2]:
plt.figure(figsize=(14, 7))
plt.grid(True)
for i in range(samples):
    plt.plot(Deltas, CPP_path[i])

plt.xlabel('Time')
plt.ylabel('CPP')

plt.savefig('CPP.png')
plt.show()

### Estimation

In [ ]:
M_kDel = np.exp(mu * Deltas + sig * W_path + CPP_path)

print(M_kDel.shape)
M_kDel

In [ ]:
D_k = np.log(M_kDel[:, 1:]) - np.log(M_kDel[:, :-1])

print(D_k.shape)
D_k

In [ ]:
def re_estimation(n_obs, i, dt, D_k, u_grid):
    cos_sq_sum = np.array([arr.sum()**2 for arr in [np.cos(u * D_k[i]) for u in u_grid]])
    sin_sq_sum = np.array([arr.sum()**2 for arr in [np.sin(u * D_k[i]) for u in u_grid]])
    re_hat = -1 / dt * np.log(n_obs) + 1 / (2 * dt) * np.log(cos_sq_sum + sin_sq_sum)
    return re_hat


def im_estimation(i, dt, D_k, u_grid):
    cos_sum = np.array([arr.sum() for arr in [np.cos(u * D_k[i]) for u in u_grid]])
    sin_sum = np.array([arr.sum() for arr in [np.sin(u * D_k[i]) for u in u_grid]])
    im_hat = (1 / dt) * np.arctan(sin_sum / cos_sum)
    return im_hat


def Lam_d(d: int, eps: float, u_til: np.ndarray) -> float:
    
    w = (eps <= u_til) & (u_til <= 1)
    
    return (w * u_til**(2 * d)).sum()


def Psi_d(
    n_obs: int, i:int, dt: float, D_k: np.ndarray,
    d: int, eps: float, U_n: float, u_til: np.ndarray 
) -> float:
    
    w = (eps <= u_til) & (u_til <= 1)
    re_hat = re_estimation(n_obs, i, dt, D_k, u_til * U_n)
    
    return (w * re_hat * (u_til * U_n)**(2 * d)).sum()


def Epsilon(
    i:int, dt: float, D_k: np.ndarray,
    eps: float, V_n: float, u_til: np.ndarray 
):
    
    w = (eps <= u_til) & (u_til <= 1)
    im_hat = im_estimation(i, dt, D_k, u_til * V_n)
    
    return (w * im_hat * u_til * V_n).sum()


def kernel(x):
    
    if abs(x) <= 0.05:
        return 1
    elif 0.05 < abs(x) and abs(x) < 1:
        return np.exp(-np.exp(-1 / (abs(x) - 0.05)) / (1 - abs(x)))
    return 0


def phi_estimation(
    i: int, n_obs: int, dt: float, u_arc: np.ndarray, D_k: np.ndarray
):
    
    return np.array([(1 / dt) * np.log((1 / n_obs) * np.sum(np.exp(1j * u * D_k[i]))) for u in u_arc])


def density_estimation(
    i: int, n_obs: int, dt: float, T_n: float, delta: float, lam_n: float, sig_n: float, mu_n: float,
    u_arc: np.ndarray, D_k: np.ndarray, x_s: float
):
    
    phi_est = phi_estimation(i, n_obs, dt, u_arc, D_k)
    kern = np.array([kernel(u / T_n) for u in u_arc])
    
    return ((T_n * delta) / (2 * np.pi * lam_n)) * np.sum(
        np.exp(-1j * u_arc * x_s) * (phi_est - 1j * mu_n * u_arc + (1 / 2) * sig_n**2 * u_arc**2 + lam_n) * kern
    )

In [ ]:
def estimation(
    mu:float, sig:float, lam:float, dt:float,
    n_obs:int, samples:int, seed:int
):
    
    # Fixing random seed
    np.random.seed(seed)
    
    u_grid1 = np.linspace(0, 10, n_obs)
    Deltas = (np.ones(shape=n_obs + 1) * dt) * np.arange(n_obs + 1)
    
    W_increments = sts.norm.rvs(loc=0, scale=np.sqrt(dt), size=(samples, n_obs))
    W_path = np.cumsum(W_increments, axis=1)
    W_path = np.insert(W_path, 0, 0, axis=1)
    
    N_increments = sts.poisson.rvs(mu=lam * dt, size=(samples, n_obs))
    N_path = np.cumsum(N_increments, axis=1)
    N_path = np.insert(N_path, 0, 0, axis=1)
    
    xi_seq = sts.norm.rvs(loc=0, scale=1, size=(samples, np.max(N_path[:, -1])))
    xi_sum = np.cumsum(xi_seq, axis=1)
    xi_sum = np.insert(xi_sum, 0, 0, axis=1)
    
    CPP_path = np.array([np.array([xi_sum[i][n_t] for n_t in N_path[i]]) for i in range(samples)])
    
    M_kDel = np.exp(mu * Deltas + sig * W_path + CPP_path)
    
    D_k = np.log(M_kDel[:, 1:]) - np.log(M_kDel[:, :-1])
    
    # Initiate for params
    eps, U_n, V_n = 0.5, 6, 6
    N = n_obs
    div = np.linspace(eps, 1, N + 1)
    
    # Lets compute u_tilda
    u_til1 = np.arange(len(div) - 1)
    u_til1 = np.vectorize(lambda i: np.random.uniform(div[i], div[i + 1]))(u_til1)
    
    #Initiate for density
    m = 10**2
    x_s = np.linspace(-5, 5, m)
    
    N = n_obs
    T_n = 3.3
    delta = 2 / N
    
    u_grid2 = np.linspace(-1, 1, N - 1)
    
    u_til2 = np.array([np.random.uniform(u_grid2[i - 1], u_grid2[i]) for i in range(1, len(u_grid2))])
    u_til2 = np.insert(u_til2, 0, -1)
    u_til2 = np.append(u_til2, 1)
    
    u_arc = T_n * u_til2
    
    sigmas_square, lambdas, mus = [], [], []
    estimated_densities = []
    
    for i in tqdm(range(samples)):
        Lam_0 = Lam_d(0, eps, u_til1)
        Lam_1 = Lam_d(1, eps, u_til1)
        Lam_2 = Lam_d(2, eps, u_til1)
    
        Psi_0 = Psi_d(n_obs, i, dt, D_k, 0, eps, U_n, u_til1)
        Psi_1 = Psi_d(n_obs, i, dt, D_k, 1, eps, U_n, u_til1)
        
        Eps = Epsilon(i, dt, D_k, eps, V_n, u_til1)
        
        sig_n = 2 * (Psi_0 * Lam_1 * U_n**2 - Psi_1 * Lam_0) / ((Lam_2 * Lam_0 - Lam_1**2) * U_n**4)
        sigmas_square.append(sig_n**2)
    
        lam_n = (Psi_1 * Lam_1 - Psi_0 * Lam_2 * U_n**2) / ((Lam_2 * Lam_0 - Lam_1**2) * U_n**2)
        lambdas.append(lam_n)
        
        mu_n = Eps / (Lam_1 * V_n**2)
        mus.append(mu_n)
        
        p_estimated = np.array([density_estimation(i, n_obs, dt, T_n, delta, lam_n, sig_n, mu_n, u_arc, D_k, x) for x in x_s])
        estimated_densities.append(p_estimated)
        
    return sigmas_square, lambdas, mus, estimated_densities

In [ ]:
sigmas_square_1000, lambdas_1000, mus_1000, est_den1000 = estimation(0, 1, 10, 0.1, 10**3, samples, seed)
sigmas_square_5000, lambdas_5000, mus_5000, est_den5000 = estimation(0, 1, 10, 0.1, 5 * 10**3, samples, seed)
sigmas_square_10000, lambdas_10000, mus_10000, est_den10000 = estimation(0, 1, 10, 0.1, 10**4, samples, seed)

In [ ]:
est_densities = [est_den1000, est_den5000, est_den10000]
density_labels = ['1000', '5000', '10000']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 10))

axes = axes.flatten()

for ax, dens_list, sample_size in zip(axes, est_densities, density_labels):
    sns.lineplot(x=x_s, y=dens_list[0], color='blue', linewidth=0.5, label='estimated', ax=ax)
    for dens in dens_list[1:]:
        sns.lineplot(x=x_s, y=dens, color='blue', linewidth=0.5, ax=ax)
        
    sns.lineplot(x=x_s, y=p_real, color='red', label='real', ax=ax)
    
    ax.set_xlabel('u')
    ax.set_ylabel('density')
    ax.set_title(f'Sample size: {sample_size}')


MSE_data = [MSE_1000, MSE_5000, MSE_10000]
sns.boxplot(data=MSE_data, ax=axes[-1])
axes[-1].axhline(y=0, color='red', linestyle='--')
axes[-1].set_xticklabels([1000, 5000, 10000])
axes[-1].set_xlabel('Sample size')
axes[-1].set_title('Boxplot of MSEs')

plt.savefig('densities.png')
plt.show()